In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString
from rtree import index
from GISio import shp2df, df2shp, get_values_at_points
from preproc import renumber_segments, get_upsegs, lines

In [2]:
newlines = '../Examples/data/added_lines.shp'
sfrlines = '../Examples/data/SFRlines2.shp'
grid = '../Examples/data/grid.shp'
domain = '../Examples/data/domain.shp'
#dem = 'D:/ATLData/BR/BadRiver/grid/dem/dem_utm_ft/w001001.adf'
dem = '/Users/aleaf/Documents/BR/BadRiver/grid/dem/dem_utm_ft/w001001.adf'

In [3]:
nl = shp2df(newlines)
sfr = shp2df(sfrlines)
tol = 100
nsegstart = sfr.segment.astype(int).max() + 1


reading ../Examples/data/added_lines.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/SFRlines2.shp...
--> building dataframe... (may take a while for large shapefiles)


In [4]:
trim_buffer=20
lns = lines(newlines, mf_grid=grid, mf_grid_node_col='node', model_domain=domain)
lns.get_end_elevs_from_dem(dem)
lns.route_lines_by_proximity()
#lns.route_lines_to_sfr(sfr, route2reach1=False)

Reading input...

reading ../Examples/data/added_lines.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/grid.shp...
--> building dataframe... (may take a while for large shapefiles)


In [9]:
nseg, outseg = lns.df.segment.values, lns.df.outseg.values

def append_upsegs(allupsegs, upsegs, nseg, outseg):
    nextupsegs = []
    for u in upsegs:
        allupsegs[u] += nseg[outseg == u].tolist()
        nextupsegs += nseg[outseg == u].tolist()
    return allupsegs, nextupsegs

# if reach data are supplied, segment/outseg pairs may be listed more than once
if len(nseg) != len(np.unique(nseg)):
    d = dict(zip(nseg, outseg))
    nseg, outseg = np.array(d.keys()), np.array(d.values())
ns = len(nseg)

allupsegs = {s: [] for s in nseg}
nexts = ns
nextupsegs = nseg[outseg == 0]
for i in range(ns):
    allupsegs, nextupsegs = append_upsegs(allupsegs, nextupsegs, nseg, outseg)
    if len(nextupsegs) == 0:
        break

In [10]:
allupsegs

{1: [],
 2: [51],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [100],
 8: [84],
 9: [25],
 10: [],
 11: [],
 12: [],
 13: [73, 78],
 14: [],
 15: [39],
 16: [80, 119],
 17: [96],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [108],
 23: [109],
 24: [],
 25: [32],
 26: [],
 27: [54],
 28: [],
 29: [116],
 30: [],
 31: [],
 32: [58],
 33: [],
 34: [30],
 35: [],
 36: [53, 102],
 37: [],
 38: [124],
 39: [79],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [118],
 50: [12, 67],
 51: [87],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [71],
 57: [],
 58: [11],
 59: [103, 121],
 60: [63],
 61: [47],
 62: [],
 63: [],
 64: [],
 65: [46, 122],
 66: [],
 67: [],
 68: [],
 69: [38],
 70: [],
 71: [5],
 72: [35],
 73: [],
 74: [66],
 75: [],
 76: [],
 77: [15],
 78: [27],
 79: [57],
 80: [],
 81: [],
 82: [],
 83: [112],
 84: [125],
 85: [],
 86: [],
 87: [],
 88: [],
 89: [],
 90: [56, 101],
 91: [],
 92: [],
 93: [34, 74],
 94: [],
 95: [],
 96: [],
 97: [],
 98: [],
 99: [

In [13]:
allupsegs

{1: [],
 2: [51, 87],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [100, 77, 15, 39, 79, 57],
 8: [84, 125],
 9: [25, 32, 58, 11],
 10: [],
 11: [],
 12: [],
 13: [73, 78, 27, 54],
 14: [],
 15: [39, 79, 57],
 16: [80, 119, 93, 34, 74, 30, 66],
 17: [96],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [108],
 23: [109, 107, 105],
 24: [],
 25: [32, 58, 11],
 26: [],
 27: [54],
 28: [],
 29: [116, 92],
 30: [],
 31: [],
 32: [58, 11],
 33: [],
 34: [30],
 35: [],
 36: [53, 102],
 37: [],
 38: [124],
 39: [79, 57],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [118],
 50: [12, 67],
 51: [87],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [71, 5],
 57: [],
 58: [11],
 59: [103, 121],
 60: [63],
 61: [47],
 62: [],
 63: [],
 64: [],
 65: [46, 122],
 66: [],
 67: [],
 68: [],
 69: [38, 124],
 70: [],
 71: [5],
 72: [35],
 73: [],
 74: [66],
 75: [],
 76: [],
 77: [15, 39, 79, 57, 79, 57, 57, 57],
 78: [27, 54],
 79: [57],
 80: [],
 81: [],
 82: [],
 83: [112],
 84: [125]

In [23]:
def append(allupsegs, k, nxt):
    vv = allupsegs.get(nxt, None)
    allupsegs[k] += nxt
    return nxt

for k, v in allupsegs.iteritems():
    print k, v
    for u in v:
        nxt = u
        while True:
            print nxt
            nxt = allupsegs.get(nxt, None)
            allupsegs[k] += nxt
            if nxt is None:
                break
        

1 []
2 [51, 87, 87, 87, 87]
51
[87]


TypeError: unhashable type: 'list'

In [5]:
nseg, outseg = lns.df.segment.values, lns.df.outseg.values
nseg, outseg

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137]),
 array([128,   0, 106,   6,  71, 104,   0,   0,   0,  24,  58,  50,   0,
         70,  77,   0,   0,   0,   0,   0, 111,   0, 114,   3,   9,  86,
         78,  86, 115,  34,   0,  25,  64,  93,  72,   0,   0,  69,  15,
      

In [21]:
upsegs = {o: set(nseg[outseg == o]) for o in np.unique(outseg)}

outsegs = outseg[outseg != 0].tolist() # exclude 0, which is the outlet designator

In [25]:
# for each outseg key, for each upseg, check for more upsegs, append until headwaters has been reached
for s in outsegs:

    up = True
    upsegslist = upsegs[s]
    for i in range(len(nseg)): # limit iterations to number of segments
        added_upsegs = set()
        [added_upsegs.update(set(upsegs[us])) for us in upsegslist if us in outsegs]
        if len(added_upsegs) == 0:
            break
        else:
            upsegslist = added_upsegs
            upsegs[s].update(added_upsegs)

# the above algorithm is recursive, so lower order streams get duplicated many times
# use a set to get unique upsegs
#upsegsd = dict([(u, list(set(upsegs[u]))) for u in outsegs])

In [28]:
upsegs[104]

{3, 4, 6, 10, 14, 24, 42, 43, 44, 62, 68, 70, 76, 81, 99, 104, 106, 120}

In [4]:
r = renumber_segments(sfr.segment.values, sfr.outseg.values)

enforcing best segment numbering...


In [5]:
sfr['segment'] = [r[s] for s in sfr.segment]
sfr['outseg'] = [r[s] for s in sfr.outseg]
sfr.sort_values(['segment', 'reach'], inplace=True)
# make sure reaches start with 1 in each segment

for s in np.unique(sfr.segment):
    df = sfr.loc[sfr.segment == s]
    sfr.loc[sfr.segment == s, 'reach'] = np.arange(1, len(df) + 1)

In [37]:
df2shp(sfr, '../Examples/data/SFRlines2.shp')

writing ../Examples/data/SFRlines2.shp...


In [13]:
start_cds = [(g.xy[0][0], g.xy[1][0]) for g in nl.geometry]
end_cds = [(g.xy[0][-1], g.xy[1][-1]) for g in nl.geometry]
sfr_start_cds = [(g.xy[0][0], g.xy[1][0]) for g in sfr.geometry]

In [14]:
max_elevs = get_values_at_points(dem, start_cds)
min_elevs = get_values_at_points(dem, end_cds)

In [15]:
nl = pd.DataFrame(nl.geometry)
nl['segment'] = np.arange(nsegstart, nsegstart+len(nl))
nl['elevMax'] = max_elevs
nl['elevMin'] = min_elevs
nl['upsegs'] = [[]] * len(nl)
nl['outseg'] = 0

In [5]:
# get index of nearest start to each end
def get_nearest(starts, ends):
    """Returns index of nearest start coordinate to each coordinate in ends.
    
    Parameters
    ----------
    starts : list of tuples
        Could be starting coordinates of each LineString.
    ends : list of tuples
        Could be ending coordinates of each LineString.
    """
    idx = index.Index()
    for i, start in enumerate(starts):
        idx.insert(i, start)
    return [list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
            else list(idx.nearest(end, 2))[1]
            for i, end in enumerate(ends)]


class lines:

    def __init__(self, shapefile, routing_tol=200):
        
        self.df = shp2df(shapefile)
        self.start_cds = [(g.xy[0][0], g.xy[1][0]) for g in self.df.geometry]
        self.end_cds = [(g.xy[0][-1], g.xy[1][-1]) for g in self.df.geometry]
        self.routing_tol = routing_tol
        
        self.df['segment'] = np.arange(1, len(self.df) + 1)
        self.df['outsegs'] = 0
        self.df['upsegs'] = [[]] * len(self.df)
        
    def get_end_elevs_from_dem(self, dem):
        
        from GISio import get_values_at_points
        self.df['elevMax'] = get_values_at_points(dem, self.start_cds)
        self.df['elevMin'] = get_values_at_points(dem, self.end_cds)
        
    def route_lines_by_proximity(self):
        
        print 'routing new lines...'
        nearest_start = get_nearest(self.start_cds, self.end_cds)

        # record the preliminary seg. number of nearest start if within tol
        self.df['outseg'] = [self.df.segment[n] if Point(*start_cds[n]).distance(Point(*end_cds[i])) < tol 
                 else 0 for i, n in enumerate(nearest_start)]

        self.df['upsegs'] = [self.df.segment[self.df.outseg == s].tolist() for s in self.df.segment]
        
    def route_lines_to_sfr(self, sfrlinework, route2reach1=True, 
                           trim_buffer=20, routing_tol=None):
        """Route the linework in the lines class to an existing
        set of lines representing an SFR package.
        
        Parameters
        ----------
        sfrlinework : str (shapefile path) or dataframe
            Contains linework representing SFR package (e.g. already broken by grid)
        route2reach1 : boolean
            If true, linework is routed to closest starting coordinate of an SFR segment.
            Otherwise, routing is to closest starting coordinate of an SFR reach 
            (existing SFR segment in that location will have to be subdivided).
        """
        tol = self.routing_tol if routing_tol is None else routing_tol
        
        if not isinstance(sfrlinework, pd.DataFrame):
            self.sfr = shp2df(sfrlinework)
        else:
            self.sfr = sfrlinework.copy()
            
        if route2reach1:
            segments = self.sfr.ix[self.sfr.reach == 1, 'segment'].tolist()
            geoms = self.sfr.ix[self.sfr.reach == 1, 'geometry'].tolist()
        else:
            segments = self.sfr.segment.tolist()
            geoms = [g for g in sfr.geometry]
        
        # update segment numbering so that it starts after highest seg in sfr dataset
        if len(set(self.df.segment).intersection(self.sfr.segment)) != 0:
            maxseg = self.sfr.segment.max()
            self.df['segment'] += maxseg
            self.df.loc[self.df.outseg > 0, 'outseg'] += maxseg
            self.df['upsegs'] = [[u + maxseg for u in us] for us in lns.df.upsegs.tolist()]
            
        sfr_start_cds = [(g.xy[0][0], g.xy[1][0]) for g in geoms]
            
        print 'routing new lines within {} to SFR...'.format(tol)

        is_outlet = self.df.outseg.values == 0
        new_lines_outlet_cds = map(tuple, np.array(self.end_cds)[is_outlet])

        # get index of nearest start to each end
        
        nearest_sfr = get_nearest(sfr_start_cds, new_lines_outlet_cds)

        # record the preliminary seg. number of nearest start if within tol
        self.df.loc[is_outlet, 'outseg'] = [segments[n] 
                                       if Point(*sfr_start_cds[n]).distance(\
                                          Point(*new_lines_outlet_cds[i])) < tol 
                                       else 0 for i, n in enumerate(nearest_sfr)]
        if not route2reach1:
            reaches = self.sfr.reach.tolist()
            self.df.loc[is_outlet, 'outreach'] = [reaches[n] 
                                       if Point(*sfr_start_cds[n]).distance(\
                                          Point(*new_lines_outlet_cds[i])) < tol 
                                       else 0 for i, n in enumerate(nearest_sfr)]
    
        def fix_newline_end(line, sfr_start_coord, trim_buffer=20):
            # trim the ends of the new lines (in case of overlap)
            # and connect them to start of closest segment or reach on SFR network
            diff = line.difference(Point(line.coords[-1]).buffer(trim_buffer))
            if diff.length > 0:
                return LineString(list(diff.coords) + [sfr_start_coord]) 
            return LineString([line.coords[0], sfr_start_coord])
        
        # only connect the lines that are within the routing tolerance
        geoms = self.df.geometry.values
        geoms[is_outlet] = [fix_newline_end(l, sfr_start_cds[nearest_sfr[i]]) 
                            if Point(l.coords[-1]).distance(\
                               Point(sfr_start_cds[nearest_sfr[i]])) < tol
                            else l 
                            for i, l in enumerate(self.df.geometry[is_outlet])]
        self.df['geometry'] = geoms

        

In [112]:
lns.df

,CARTO_USE_,CREATE_DAT,ENABLED,FLIP_CHANG,GEOM_CHANG,HYDROCODE,HYDROID,HYDROTYPE,IN_STATE_C,LANDLOCK_C,...,WATERBODY_,WBIC_CHANG,geometry,segment,outsegs,upsegs,elevMax,elevMin,outseg,outreach
0,1,None,1,None,None,110,200199992,508,1,0,...,0,None,LINESTRING (697207.5139276112 5144886.68895074...,109,0,[],1327.925293,1333.011230,236,NaN
1,1,None,1,None,None,110,200199417,508,1,0,...,0,None,LINESTRING (695521.2316259361 5141918.46333385...,110,0,[159],1364.973267,1319.599243,65,65
2,0,None,1,None,None,120,200198915,504,1,0,...,0,None,LINESTRING (692732.9645041943 5139590.03203743...,111,0,"[132, 189]",1314.611328,1313.467041,214,NaN
3,0,None,1,None,None,120,200198851,504,1,0,...,0,None,LINESTRING (692250.9203589084 5139196.41594549...,112,0,[184],1310.088013,1305.923462,114,NaN
4,1,None,1,None,None,110,200199018,508,1,0,...,0,None,LINESTRING (687936.6251382845 5139948.55256021...,113,0,[],1272.710205,1246.875000,179,NaN
5,1,None,1,None,None,100,200198850,508,1,0,...,0,None,LINESTRING (692249.5093258156 5139241.06098325...,114,0,"[112, 151]",1305.923462,1299.937988,212,NaN
6,0,None,1,None,None,120,200198543,504,1,0,...,0,None,LINESTRING (696169.9742699204 5137959.01249069...,115,0,[208],1459.119995,1454.165894,3,37
7,1,None,1,None,None,110,200199280,508,1,0,...,0,None,LINESTRING (697811.0094337247 5141625.95269692...,116,0,[192],1399.471313,1351.638550,65,15
8,1,None,1,None,None,100,200199179,508,1,0,...,0,None,LINESTRING (694189.4015423749 5140853.67015157...,117,0,[133],1350.218872,1344.326050,44,15
9,1,None,1,None,None,110,200198878,508,1,0,...,0,None,LINESTRING (692904.9004312663 5139214.42371773...,118,0,[152],1386.414673,1316.827515,132,NaN


In [114]:
g = lns.df.ix[lns.df.segment == 163, 'geometry'].values[0]
lns.df.ix[lns.df.segment == 163]
Point(g.coords[-1]).distance(Point(sfr_start_cds[nearest_sfr[54]]))

5760.955430424282

In [6]:
trim_buffer=20
lns = lines(newlines, mf_grid=grid)
lns.get_end_elevs_from_dem(dem)
lns.route_lines_by_proximity()
lns.route_lines_to_sfr(sfr, route2reach1=False)

Reading input...

reading ../Examples/data/added_lines.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/grid.shp...
--> building dataframe... (may take a while for large shapefiles)
setting model domain to extent of grid by performing unary union of grid cell geometries...
(may take a few minutes for large grids)


KeyError: 'elevs'

In [120]:
df2shp(lns.df, 'temp/junk.shp', epsg=26715)

writing temp/junk.shp...


In [1]:
lns.df

NameError: name 'lns' is not defined

In [67]:
lns.df.ix[is_outlet, 'segment']

54     163
59     168
94     203
97     206
127    236
131    240
134    243
Name: segment, dtype: int64

In [88]:
sfr.segment

0        733
1        733
2        733
3        733
4        733
5        733
6        733
7        733
8        733
9        733
10       733
11       733
12       733
13       733
14       733
15       733
16       733
17       733
18       733
19       733
20       733
21       737
22       737
23       737
24       737
25       737
26       737
27       737
28       737
29       737
        ... 
1565    1139
1566    1139
1567    1139
1568    1139
1569    1139
1570    1139
1571    1139
1572    1139
1573    1139
1574    1139
1575    1139
1576    1139
1577    1139
1578    1139
1579    1139
1580    1139
1581    1139
1582    1139
1583    1139
1584    1139
1585    1139
1586    1139
1587    1228
1588    1228
1589    1228
1590    1228
1591    1228
1592    1228
1593    1228
1594    1228
Name: segment, dtype: int64

In [142]:
# get index of nearest start to each end
def get_nearest(starts, ends):
    idx = index.Index()
    for i, start in enumerate(starts):
        idx.insert(i, start)
    return [list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
            else list(idx.nearest(end, 2))[1]
            for i, end in enumerate(ends)]

#def route_lines_by_proximity(start_crds, end_crds)
print 'routing new lines...'
nearest_start = get_nearest(start_cds, end_cds)

# record the preliminary seg. number of nearest start if within tol
nl['outseg'] = [nl.segment[n] if Point(*start_cds[n]).distance(Point(*end_cds[i])) < tol 
                 else 0 for i, n in enumerate(nearest_start)]

nl['upsegs'] = [nl.segment[nl.outseg == s].tolist() for s in nl.segment]

routing new lines...


AttributeError: 'DataFrame' object has no attribute 'segment'

In [15]:
print 'routing new lines to SFR...'

is_outlet = nl.outseg.values == 0
new_lines_outlet_cds = map(tuple, np.array(end_cds)[is_outlet])

# get index of nearest start to each end
nearest_sfr = get_nearest(sfr_start_cds, new_lines_outlet_cds)

# record the preliminary seg. number of nearest start if within tol
nl.loc[is_outlet, 'outseg'] = [sfr.segment[n] 
                               if Point(*sfr_start_cds[n]).distance(\
                                  Point(*new_lines_outlet_cds[i])) < tol 
                               else 0 for i, n in enumerate(nearest_sfr)]

routing new lines to SFR...


In [17]:
df2shp(nl, 'temp/junk.shp', epsg=26715)

writing temp/junk.shp...


In [ ]:
[Point(*sfr_start_cds[n]).distance(\
Point(*new_lines_outlet_cds[i])) 
for i, n in enumerate(nearest_sfr)]

In [ ]:
idx = index.Index()
for i, start in enumerate(start_cds):
    idx.insert(i, start)
[list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
        else list(idx.nearest(end, 2))[1]
        for i, end in enumerate(end_cds)]

In [ ]:
[i for i, end in enumerate(end_cds)]